In [97]:
import torch
from evogp.tree import Forest
from torch import Tensor

a = torch.arange(1, 18)
b = torch.arange(1, 18)
c = torch.stack([a, b], dim=0)

start = torch.tensor([2, 5])
end = torch.tensor([4, 6])


def subtensor(tensor: torch.Tensor, start, length):
    start = start.reshape(-1, 1)
    length = length.reshape(-1, 1)
    end = start + length
    max_len = torch.max(length)
    indices = torch.arange(max_len) + start
    shifted_tensor = tensor.gather(1, indices)
    return torch.where(indices < end, shifted_tensor, 0)


def vmap_subtree(forest: Forest, pos):
    pos = pos.reshape(-1, 1)
    length = forest.batch_subtree_size.gather(1, pos)
    Forest(
        forest.input_len,
        forest.output_len,
        subtensor(forest.batch_node_value, pos, length),
        subtensor(forest.batch_node_type, pos, length),
        subtensor(forest.batch_subtree_size, pos, length),
    )


result = subtensor(c, start, end)

print(result)

tensor([[ 3,  4,  5,  6,  0,  0],
        [ 6,  7,  8,  9, 10, 11]])


In [ ]:
import torch

# 示例输入
num_mutate = 5
mutate_positions = torch.tensor([0, 1, 2, 3, 4])
size_tensor = torch.tensor([
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 6, 7],
    [4, 5, 6, 7, 8],
    [5, 6, 7, 8, 9]
])
nth_childs = torch.tensor([2, 3, 4, 1, 5])  # 指定每个位置跳跃的次数

# 初始化 pos，第一跳为 mutate_positions + 1
pos = mutate_positions + 1
print(pos)

# 循环跳跃 nth_childs.max() 次
for i in range(2, nth_childs.max() + 1):
    jump = size_tensor[torch.arange(num_mutate), pos]  # 计算当前跳跃的步长
    pos = torch.where(nth_childs >= i, pos + jump, pos)  # 更新 pos，仅对需要跳跃的位置进行累加

print(pos)


tensor([1, 2, 3, 4, 5])


IndexError: index 5 is out of bounds for dimension 1 with size 5

In [83]:
import torch

tree_size = torch.tensor(1)

print(a)

torch.Size([])


In [94]:
a = torch.rand(4, 4)
print(a)
torch.argmax(a, 0)

tensor([[0.4553, 0.5184, 0.1132, 0.0275],
        [0.2178, 0.5921, 0.8125, 0.2033],
        [0.1443, 0.4339, 0.9722, 0.1025],
        [0.5645, 0.8002, 0.5949, 0.1533]])


tensor([3, 3, 2, 1])

In [89]:
import torch

def process_row(row: torch.Tensor, init_idx: int, jumps: int) -> int:
    """
    模拟跳跃操作，给定起始索引 init_idx 和跳跃次数 jumps，
    根据 row 中的跳跃值依次累加，返回最终索引。
    
    参数：
    - row: Tensor，包含跳跃值
    - init_idx: int，初始索引
    - jumps: int，跳跃次数

    返回：
    - final_idx: int，最终索引
    """
    final_idx = init_idx
    for _ in range(jumps):
        final_idx += row[final_idx].item()  # 每次跳跃累加 row 中的值
    return final_idx


TypeError: randint() received an invalid combination of arguments - got (int, Tensor, tuple), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (int high, tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (int low, int high, tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)


In [6]:
from tree import Tree, Forest

generate_configs = {
    "seed": 1,
    "gp_len": 64,
    "input_len": 3,
    "output_len": 1,
    "const_prob": 0.5,
    "out_prob": 0.5,
    "func_prob": {"+": 0.25, "-": 0.25, "*": 0.25, "/": 0.25},
    "max_layer_cnt": 5,
    "layer_leaf_prob": 0.2,
    "const_range": (-1, 1),
    "sample_cnt": 8
}

tree = Forest.random_generate(pop_size=10, **generate_configs)

SyntaxError: invalid character '🌳' (U+1F333) (280915712.py, line 17)

In [2]:
print(forest)

[
  x[0] , 
  0.70 , 
  / / * + - 0.70 0.70 + x[0] 0.70 / - 0.70 -0.43 + 0.22 0.22 * / - 0.91 x[0] / -0.62 x[0] / / -0.43 x[0] + 0.22 -0.43 + -0.43 / + - x[0] x[2] * x[0] x[2] x[1] , 
  * x[0] * * + 0.70 - x[2] -0.22 - / x[0] x[0] + x[0] -0.22 / * / x[2] -0.62 x[2] * - x[1] x[0] x[2] , 
  + - - 0.91 -0.22 -0.43 + * * * x[1] x[2] + 0.70 x[2] + - -0.22 x[1] * x[2] x[2] / + * 0.22 0.91 + -0.62 x[0] x[2] , 
  + - * - - 0.70 x[2] + -0.62 0.22 / + x[1] -0.43 -0.62 + + + 1.00 x[0] / x[2] 1.00 - + 0.70 x[2] 0.70 - / 1.00 - / x[1] 0.91 1.00 * / * -0.43 1.00 - x[0] 0.91 + / x[1] x[2] x[0] , 
  * / - / / -0.22 1.00 / x[2] x[2] - x[1] * x[2] x[0] * + - 0.22 x[2] - x[1] x[2] - + -0.62 x[0] + x[0] 0.04 * * - * x[2] x[2] -0.62 * + x[0] x[0] * x[2] x[1] - / * -0.43 0.70 / 1.00 x[0] / -0.62 0.70 , 
  x[0] , 
  -0.43 , 
  * / / - 0.04 -0.43 x[0] / / / 0.91 x[1] * x[0] 0.70 0.91 / / x[0] -0.62 - * * 1.00 x[2] * x[1] -0.43 + x[1] / 0.70 x[1] , 
]


In [3]:
import torch
tree = forest[0]
res = tree.forward(torch.Tensor([1, 2, 3]))
print(res)

tensor([1.], device='cuda:0')


In [4]:
res = forest.forward(torch.ones(10, 3))

In [5]:
res

tensor([[ 1.0000e+00],
        [ 6.9722e-01],
        [ 2.4932e+01],
        [-4.2417e+04],
        [ 1.7681e+00],
        [ 1.5527e+01],
        [-8.2777e-01],
        [ 1.0000e+00],
        [-4.3255e-01],
        [ 2.4820e-01]], device='cuda:0')

In [3]:
import torch
a = torch.tensor([1,2,3,4,5,6,7,8,9])
bool_indices = a % 2 == 0

a, bool_indices

(tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 tensor([False,  True, False,  True, False,  True, False,  True, False]))

In [6]:
b = a[bool_indices] + 100
b

tensor([102, 104, 106, 108])

In [7]:
a[bool_indices] = b

In [8]:
a

tensor([  1, 102,   3, 104,   5, 106,   7, 108,   9])

In [16]:
import torch
a = torch.tensor([1,2,3,4,5,6,7,8,9], dtype=torch.int16, device="cuda")
torch.gather(a, 0, torch.tensor([1 ,2, 3], dtype=torch.int64, device="cuda"))

RuntimeError: "cuda_scatter_gather_base_kernel_func" not implemented for 'UInt16'

In [1]:
import sys
print(sys.getdefaultencoding())  # 应该输出 'utf-8'

utf-8


In [3]:
"&#128512; &#128526; &#128640; &#10084;"

'&#128512; &#128526; &#128640; &#10084;'

In [5]:
a = "😀 😎 🚀 ❤️"
a

'😀 😎 🚀 ❤️'

In [7]:
code = "😀😎🚀❤️ = 12342435"
exec(code)

SyntaxError: invalid character '😀' (U+1F600) (<string>, line 1)